In [7]:
!pip install mediapipe==0.10.20 
!pip install opencv-python numpy tensorflow scikit-learn

In [8]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

class EnhancedFaceDetector:
    def __init__(self):
        """
        Initialize the Enhanced Face Detection model with key components
        """
        # MediaPipe face detection setup
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_drawing = mp.solutions.drawing_utils
        
        # Model parameters
        self.input_shape = (224, 224, 3)  # Standard image size
        self.model = None

    def load_dataset(self, image_dir, mask_dir=None):
        """
        Load images from local directories
        
        Args:
            image_dir (str): Path to directory containing face images
            mask_dir (str, optional): Path to directory containing mask images
        
        Returns:
            tuple: Loaded images and labels
        """
        images = []
        labels = []
        
        # Load images
        for filename in os.listdir(image_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(image_dir, filename)
                img = cv2.imread(img_path)
                
                # Preprocess image
                img = cv2.resize(img, (self.input_shape[0], self.input_shape[1]))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = img / 255.0  # Normalize pixel values
                
                images.append(img)
                
                # Simple binary label - face present or not
                labels.append(1 if self._detect_face_mediapipe(img) else 0)
        
        return np.array(images), np.array(labels)

    def _detect_face_mediapipe(self, image):
        """
        Use MediaPipe to detect if a face is present
        
        Args:
            image (np.array): Input image
        
        Returns:
            bool: True if face detected, False otherwise
        """
        with self.mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
            results = face_detection.process(image)
            return results.detections is not None and len(results.detections) > 0

    def create_model(self):
        """
        Create a convolutional neural network for face detection
        
        Returns:
            tf.keras.Model: Compiled face detection model
        """
        model = Sequential([
            # Convolutional layers for feature extraction
            Conv2D(32, (3, 3), activation='relu', input_shape=self.input_shape),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            
            # Flatten and dense layers for classification
            Flatten(),
            Dense(64, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')  # Binary classification
        ])
        
        # Compile the model
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model

    def train_model(self, image_dir, validation_split=0.2, epochs=10):
        """
        Train the face detection model
        
        Args:
            image_dir (str): Directory containing training images
            validation_split (float): Percentage of data for validation
            epochs (int): Number of training epochs
        """
        # Load dataset
        X, y = self.load_dataset(image_dir)
        
        # Split data
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=validation_split, random_state=42
        )
        
        # Data augmentation
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )
        
        # Create and train model
        self.model = self.create_model()
        
        # Training with data augmentation
        self.model.fit(
            datagen.flow(X_train, y_train, batch_size=32),
            validation_data=(X_val, y_val),
            epochs=epochs
        )

    def save_model(self, output_path='face_detection_model.pickle'):
        """
        Save the trained model to a pickle file
        
        Args:
            output_path (str): Path to save the model
        """
        if self.model is None:
            raise ValueError("Model has not been trained yet!")
        
        with open(output_path, 'wb') as f:
            pickle.dump({
                'model': self.model,
                'input_shape': self.input_shape
            }, f)
        
        print(f"Model saved to {output_path}")

    def load_saved_model(self, model_path='face_detection_model.pickle'):
        """
        Load a previously saved model
        
        Args:
            model_path (str): Path to the saved model file
        
        Returns:
            object: Loaded model
        """
        with open(model_path, 'rb') as f:
            saved_data = pickle.load(f)
        
        self.model = saved_data['model']
        self.input_shape = saved_data['input_shape']
        
        print("Model loaded successfully!")
        return self.model

    def predict(self, image_path):
        """
        Predict face detection on a single image
        
        Args:
            image_path (str): Path to the image
        
        Returns:
            tuple: Prediction result and confidence
        """
        # Load and preprocess image
        img = cv2.imread(image_path)
        img = cv2.resize(img, (self.input_shape[0], self.input_shape[1]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        
        # Predict
        prediction = self.model.predict(img)[0][0]
        return (prediction > 0.5, prediction)

# Usage Example
def main():
    # Initialize the face detector
    face_detector = EnhancedFaceDetector()
    
    # Training
    # Specify the path to your local image directory
    training_image_dir = 'C:/Users/User/Downloads/Face_Recognition dataset/train/images'
    face_detector.train_model(training_image_dir)
    
    # Save the model
    face_detector.save_model('enhanced_face_detection_model.pickle')
    
    # Load and test the model
    loaded_model = face_detector.load_saved_model('enhanced_face_detection_model.pickle')
    
    # Predict on a new image
    test_image_path = 'C:/Users/User/Downloads/Face_Recognition dataset/train/images/0133_4.jpg'
    result, confidence = face_detector.predict(test_image_path)
    print(f"Face Detected: {result}, Confidence: {confidence}")

if __name__ == "__main__":
    main()

ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.